In [15]:
from crewai import Agent, LLM
from dotenv import load_dotenv
import os
load_dotenv()  # Load environment variables from .env file
#os.environ["OPENAI_API_KEY"] = os.getenv("GEMINI_API_KEY")


llm = LLM(model="gemini/gemini-2.0-flash", verbose=True, temperature=0.5,
          api_key=os.getenv("GEMINI_API_KEY"))

## Guardrails Demo 1

In [16]:
def validate_summary_length(task_output):
    try:
        print("Validating summary length")
        task_str_output = str(task_output)
        total_words = len(task_str_output.split())

        print(f"Word count: {total_words}")

        if total_words > 150:
            print("Summary exceeds 150 words")
            return (False, f"Summary exceeds 150 words. Word count: {total_words}")

        if total_words == 0:
            print("Summary is empty")
            return (False, "Generated summary is empty.")

        print("Summary is valid")
        return (True, task_output)

    except Exception as e:
        print("Validation system error")
        return (False, f"Validation system error: {str(e)}")


In [19]:
from crewai import Task, Agent

summary_agent = Agent(
    role="Summary Agent",
    goal="Summarize the research paper 'Convolutional Neural Networks' in 150 words.",
    backstory="You are a specialized agent that summarizes research papers.",
    verbose=True,
    llm=llm
)

summary_task = Task(
    description="Summarize a research paper in 150 words.",
    expected_output="A concise research summary 150 words.",
    agent=summary_agent,
    guardrail=validate_summary_length,

    max_retries=3
)


In [20]:
from crewai import Crew

summary_crew = Crew(
    agents=[summary_agent],
    tasks=[summary_task],
    verbose=True
)

result = summary_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 5a6e4b83-ba5e-440f-9596-faefb9629b2a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 21c904cc-e6b7-4f5b-b0d5-fcf3f11c27de
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 21c904cc-e6b7-4f5b-b0d5-fcf3f11c27de
       Status: Executing Task...
    └── 🤖 Agent: Summary Agent
            Status: In Progress

# Agent: Summary Agent
## Task: Summarize a research paper in 150 words.


🚀 Crew: crew
└── 📋 Task: 21c904cc-e6b7-4f5b-b0d5-fcf3f11c27de
       Status: Executing Task...
    └── 🤖 Agent: Summary Agent
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: 21c904cc-e6b7-4f5b-b0d5-fcf3f11c27de
       Status: Executing Task...
    └── 🤖 Agent: Summary Agent
            Status: ✅ Completed

Validating summary length
Word count: 119
Summary is valid


🚀 Crew: crew
└── 📋 Task: 21c904cc-e6b7-4f5b-b0d5-fcf3f11c27de
       Assigned to: Summary Agent
       Status: ✅ Completed
    └── 🤖 Agent: Summary Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 21c904cc-e6b7-4f5b-b0d5-fcf3f11c27de                                                                     │
│  Agent: Summary Agent                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 5a6e4b83-ba5e-440f-9596-faefb9629b2a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

### Ask the agent to write a 200 word summary instead and notice the guardrail output:

In [21]:
from crewai import Task, Agent

summary_agent = Agent(
    role="Summary Agent",
    goal="Summarize the research paper 'Convolutional Neural Networks' in 200 words.",
    backstory="You are a specialized agent that summarizes research papers.",
    verbose=True,
    llm=llm
)

summary_task = Task(
    description="Summarize a research paper in 200 words.",
    expected_output="A concise research summary 200 words.",
    agent=summary_agent,
    guardrail=validate_summary_length,
    max_retries=3
)


In [22]:
from crewai import Crew

summary_crew = Crew(
    agents=[summary_agent],
    tasks=[summary_task],
    verbose=True
)

result = summary_crew.kickoff()

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 459d23f1-2157-4011-a8b7-b050092a7e7a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6
       Status: Executing Task...
    └── 🤖 Agent: Summary Agent
            Status: In Progress

# Agent: Summary Agent
## Task: Summarize a research paper in 200 words.


🚀 Crew: crew
└── 📋 Task: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6
       Status: Executing Task...
    └── 🤖 Agent: Summary Agent
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6
│      Status: Executing Task...
│   └── 🤖 Agent: Summary Agent
│           Status: ✅ Completed
└── 📋 Task: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6
       Status: Executing Task...
    └── 🤖 Agent: Summary Agent
            Status: ✅ Completed

Validating summary length
Word count: 134
Summary is valid


🚀 Crew: crew
├── 📋 Task: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6
│      Assigned to: Summary Agent
│      Status: ✅ Completed
│   └── 🤖 Agent: Summary Agent
│           Status: ✅ Completed
└── 📋 Task: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6
       Status: Executing Task...
    └── 🤖 Agent: Summary Agent
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 42fad433-377a-4d7f-9aaf-0a1af9ab35b6                                                                     │
│  Agent: Summary Agent                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 459d23f1-2157-4011-a8b7-b050092a7e7a                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

## Guardrails demo 2

In [23]:
from pydantic import BaseModel

class ResearchReport(BaseModel):
    """Represents a structured research report"""
    title: str
    summary: str
    key_findings: list[str]

In [24]:
import json
from typing import Tuple, Any

def validate_json_report(result: str) -> Tuple[bool, Any]:
    """Ensures AI-generated output is valid JSON with required fields."""
    try:
        # Parse JSON output
        data = json.loads(result.pydantic.model_dump_json())

        # Check required fields
        if "title" not in data or "summary" not in data or "key_findings" not in data:
            return (False, "Missing required fields: title, summary, or key_findings.")

        return (True, result)  # JSON is valid
    except json.JSONDecodeError:
        return (False, "Invalid JSON format. Please ensure correct syntax.")


In [25]:
from crewai import Agent

# Create the AI Agent
research_report_agent = Agent(
    role="Research Analyst",
    goal="Generate structured JSON reports for research papers",
    backstory="You are an expert in technical writing and structured reporting.",
    verbose=False,
    llm=llm
)


In [26]:
from crewai import Task

research_report_task = Task(
    description="Generate a structured research report in valid JSON format.",
    expected_output="A valid JSON object containing 'title', 'summary', and 'key_findings'.",
    agent=research_report_agent,
    output_pydantic=ResearchReport,  # Ensures structured output
    guardrail=validate_json_report,  # Validate output before passing to next step
    max_retries=3  # Allow up to 3 retries if validation fails
)

In [27]:
from crewai import Crew

research_crew = Crew(
    agents=[research_report_agent],
    tasks=[research_report_task],
    verbose=True  # Display execution details
)


In [28]:
result = research_crew.kickoff()

# Display the validated JSON output
print("Final Research Report:", result.pydantic)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 08ca242f-ecf9-4260-b2a5-fc3806dd878e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 5dfb6a6c-ca91-4a73-95c0-3769ca027598
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 5dfb6a6c-ca91-4a73-95c0-3769ca027598
       Status: Executing Task...
    └── 🤖 Agent: Research Analyst
            Status: In Progress

# Agent: Research Analyst
## Task: Generate a structured research report in valid JSON format.


🚀 Crew: crew
└── 📋 Task: 5dfb6a6c-ca91-4a73-95c0-3769ca027598
       Status: Executing Task...
    └── 🤖 Agent: Research Analyst
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
└── 📋 Task: 5dfb6a6c-ca91-4a73-95c0-3769ca027598
       Status: Executing Task...
    └── 🤖 Agent: Research Analyst
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 5dfb6a6c-ca91-4a73-95c0-3769ca027598
       Assigned to: Research Analyst
       Status: ✅ Completed
    └── 🤖 Agent: Research Analyst
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 5dfb6a6c-ca91-4a73-95c0-3769ca027598                                                                     │
│  Agent: Research Analyst                                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 08ca242f-ecf9-4260-b2a5-fc3806dd878e                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Final Research Report: title='The Impact of Artificial Intelligence on Healthcare' summary='This research investigates the transformative impact of artificial intelligence (AI) on the healthcare industry. It explores various applications of AI, including diagnostics, drug discovery, personalized medicine, and patient care. The study analyzes the benefits, challenges, and ethical considerations associated with AI adoption in healthcare, providing insights into its potential to improve patient outcomes, reduce costs, and enhance the overall efficiency of healthcare systems.' key_findings=["AI-powered diagnostic tools demonstrate improved accuracy and speed in detecting diseases such as cancer and Alzheimer's.", 'AI algorithms accelerate drug discovery by identifying potential drug candidates and predicting their efficacy.', 'Personalized medicine, driven by AI, enables tailored treatment plans based on individual patient characteristics and genetic profiles.', 'AI-enabled robotic surgery

## Reference other tasks

In [29]:
from pydantic import BaseModel

class ResearchFindings(BaseModel):
    """Structured research report output"""
    title: str
    key_findings: list[str]

class AnalysisSummary(BaseModel):
    """Structured summary of research findings"""
    insights: list[str]
    key_takeaways: str


In [30]:
from crewai import Agent, LLM

# AI Agents
research_agent = Agent(
    role="AI Researcher",
    goal="Find and summarize the latest AI advancements",
    backstory="You are an expert AI researcher who stays up to date with the latest innovations.",
    verbose=True,
    llm=llm
)

analysis_agent = Agent(
    role="AI Analyst",
    goal="Analyze AI research findings and extract key insights",
    backstory="You are a data analyst who extracts valuable insights from research data.",
    verbose=True,
    llm=llm
)

writer_agent = Agent(
    role="Tech Writer",
    goal="Write a well-structured blog post on AI trends",
    backstory="You are a technology writer skilled at transforming complex AI research into readable content.",
    verbose=True,
    llm=llm
)


In [31]:
from crewai import Task

# Step 1: Research Task
research_task = Task(
    description="Find and summarize the latest AI advancements",
    expected_output="A structured list of recent AI breakthroughs",
    agent=research_agent,
    output_pydantic=ResearchFindings  # Structured output
)

# Step 2: Analysis Task (References Research Task Output)
analysis_task = Task(
    description="Analyze AI research findings and extract key insights",
    expected_output="A structured summary with key takeaways",
    agent=analysis_agent,
    output_pydantic=AnalysisSummary,
    context=[research_task]  # Receives output from research_task
)

# Step 3: Blog Writing Task (References Both Research and Analysis)
blog_writing_task = Task(
    description="Write a detailed blog post about AI trends",
    expected_output="A well-structured blog post",
    agent=writer_agent,
    context=[research_task, analysis_task]  # Uses both research and analysis outputs
)


In [32]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[research_agent, analysis_agent, writer_agent],
    tasks=[research_task, analysis_task, blog_writing_task],
    verbose=True
)

# Execute the workflow
result = ai_research_crew.kickoff()

# Print the final blog post
print("\n=== Generated Blog Post ===")
print(result.raw)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2e4e8edd-6f8b-496e-8889-6967b5533e02                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 81e56d63-d43d-4888-8d68-a0ab3938ab82
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 81e56d63-d43d-4888-8d68-a0ab3938ab82
       Status: Executing Task...
    └── 🤖 Agent: AI Researcher
            Status: In Progress

# Agent: AI Researcher
## Task: Find and summarize the latest AI advancements


🚀 Crew: crew
└── 📋 Task: 81e56d63-d43d-4888-8d68-a0ab3938ab82
       Status: Executing Task...
    └── 🤖 Agent: AI Researcher
            Status: In Progress
        └── 🧠 Thinking...

...

🚀 Crew: crew
├── 📋 Task: 81e56d63-d43d-4888-8d68-a0ab3938ab82
│      Assigned to: AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Researcher
│           Status: ✅ Completed
├── 📋 Task: be84ad0a-1820-41dc-b7dc-633afda68188
│      Assigned to: AI Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Analyst
│           Status: ✅ Completed
└── 📋 Task: 7940154e-c955-4247-bdff-dca845ca6b9d
       Status: Executing Task...
    └── 🤖 Agent: Tech Writer
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 81e56d63-d43d-4888-8d68-a0ab3938ab82
│      Assigned to: AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Researcher
│           Status: ✅ Completed
├── 📋 Task: be84ad0a-1820-41dc-b7dc-633afda68188
│      Assigned to: AI Analyst
│      Status: ✅ Completed
│   └── 🤖 Agent: AI Analyst
│           Status: ✅ Completed
└── 📋 Task: 7940154e-c955-4247-bdff-dca845ca6b9d
       Assigned to: Tech Writer
       Status: ✅ Completed
    └── 🤖 Agent: Tech Writer
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 7940154e-c955-4247-bdff-dca845ca6b9d                                                                     │
│  Agent: Tech Writer                                                                                             │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2e4e8edd-6f8b-496e-8889-6967b5533e02                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


=== Generated Blog Post ===
## Riding the Wave: Top AI Trends Shaping the Future

Artificial intelligence is no longer a futuristic fantasy; it's a rapidly evolving reality reshaping industries and redefining what's possible. From generating stunningly realistic videos from text prompts to processing massive amounts of information with unprecedented speed, the pace of innovation in AI is staggering. In this post, we'll dive into some of the most exciting AI trends currently making waves, exploring the breakthroughs that are pushing the boundaries of what AI can achieve.

### Gemini 1.5: A Giant Leap in Long-Context Understanding

Google's Gemini 1.5 is making headlines for its remarkable ability to process vast amounts of information. One of its most significant achievements is its significantly improved long-context understanding, capable of processing up to 1 million tokens. This allows the model to analyze and understand incredibly complex and nuanced information, opening doors to 

In [33]:
analysis_task.output

TaskOutput(description='Analyze AI research findings and extract key insights', name=None, expected_output='A structured summary with key takeaways', summary='Analyze AI research findings and extract key insights...', raw='```json\n{\n  "insights": [\n    "Gemini 1.5 showcases significant advancements in long-context understanding and multimodal processing.",\n    "Groq\'s LPU Inference Engine achieves unprecedented inference speed and efficiency for large language models.",\n    "Sora demonstrates remarkable progress in text-to-video generation, creating realistic and imaginative videos.",\n    "Diffusion models continue to improve in image and video generation quality, control, and efficiency.",\n    "Reinforcement learning is making strides in robustness, sample efficiency, and real-world applications."\n  ],\n  "key_takeaways": "The AI research landscape is rapidly evolving, with notable progress in areas such as large language models, efficient inference, generative models, and re

In [34]:
analysis_task.output.pydantic

AnalysisSummary(insights=['Gemini 1.5 showcases significant advancements in long-context understanding and multimodal processing.', "Groq's LPU Inference Engine achieves unprecedented inference speed and efficiency for large language models.", 'Sora demonstrates remarkable progress in text-to-video generation, creating realistic and imaginative videos.', 'Diffusion models continue to improve in image and video generation quality, control, and efficiency.', 'Reinforcement learning is making strides in robustness, sample efficiency, and real-world applications.'], key_takeaways='The AI research landscape is rapidly evolving, with notable progress in areas such as large language models, efficient inference, generative models, and reinforcement learning. These advancements are paving the way for more powerful, efficient, and versatile AI systems with the potential to transform various industries and aspects of human life.')

## Async Execution

In [35]:
from pydantic import BaseModel

class AIResearchFindings(BaseModel):
    """Represents structured research on AI breakthroughs."""
    title: str
    key_findings: list[str]

class AIRegulationFindings(BaseModel):
    """Represents structured research on AI regulations."""
    region: str
    key_policies: list[str]

class FinalAIReport(BaseModel):
    """Combines AI research & regulation analysis into a report."""
    executive_summary: str
    key_trends: list[str]


In [36]:
from crewai import Agent

# Researcher for AI breakthroughs
research_agent = Agent(
    role="AI Researcher",
    goal="Find and summarize the latest AI breakthroughs",
    backstory="An expert AI researcher who tracks technological advancements.",
    verbose=True,
    llm=llm
)

# Analyst for AI regulations
regulation_agent = Agent(
    role="AI Policy Analyst",
    goal="Analyze global AI regulations and summarize policies",
    backstory="A government policy expert specializing in AI ethics and laws.",
    verbose=True,
    llm=llm
)

# Writer for the final AI report
writer_agent = Agent(
    role="AI Report Writer",
    goal="Write a structured report combining AI breakthroughs and regulations",
    backstory="A professional technical writer who crafts AI research reports.",
    verbose=True,
    llm=llm
)


In [37]:
from crewai import Task

# Task 1: AI Breakthroughs Research (Asynchronous)
research_ai_task = Task(
    description="Research the latest AI advancements and summarize key breakthroughs.",
    expected_output="A structured list of AI breakthroughs.",
    agent=research_agent,
    output_pydantic=AIResearchFindings,
    async_execution=True  # Runs asynchronously
)

# Task 2: AI Regulation Analysis (Asynchronous)
research_regulation_task = Task(
    description="Analyze the latest AI regulations worldwide and summarize key policies.",
    expected_output="A structured summary of AI regulations by region.",
    agent=regulation_agent,
    output_pydantic=AIRegulationFindings,
    async_execution=True  # Runs asynchronously
)

# Task 3: Generate AI Research Report (Depends on the first two tasks)
generate_report_task = Task(
    description="Write a structured report summarizing AI breakthroughs and regulations.",
    expected_output="A final AI report summarizing both aspects.",
    agent=writer_agent,
    output_pydantic=FinalAIReport,
    context=[research_ai_task, research_regulation_task]  # Waits for these tasks to complete
)


In [38]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[research_agent, regulation_agent, writer_agent],
    tasks=[research_ai_task, research_regulation_task, generate_report_task],
    verbose=True
)

# Execute the workflow
result = ai_research_crew.kickoff()

# Print the final AI report
print("\n=== Generated AI Report ===")
print(result.raw)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 97fde09d-e4be-4a45-b45e-00d3b5e92812                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: f8fdc01a-00a3-49a7-bcc6-301c9af063af
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: f8fdc01a-00a3-49a7-bcc6-301c9af063af
│      Status: Executing Task...
└── 📋 Task: e28d943f-ad54-4713-b79f-bb221005ea07
       Status: Executing Task...

🚀 Crew: crew
├── 📋 Task: f8fdc01a-00a3-49a7-bcc6-301c9af063af
│      Status: Executing Task...
└── 📋 Task: e28d943f-ad54-4713-b79f-bb221005ea07
       Status: Executing Task...
    └── 🤖 Agent: AI Researcher
            Status: In Progress

# Agent: AI Researcher
## Task: Research the latest AI advancements and summarize key breakthroughs.


...

Task Completed!
              Task: Find and summarize the latest AI breakthroughs from the last week.
              Output Summary: Find and summarize the latest AI breakthroughs from the last...
News summary saved to latest_ai_news.txt
# Agent: Project Research Manager
## Task: Analyze the market demand for the project title 'Multi-Agent System'
 

I encountered an error while trying to use the tool. This was the error: unhashable type: 'dict'.
 Tool Delegate work to coworker accepts these inputs: Tool Name: Delegate work to coworker
Tool Arguments: {'task': {'description': 'The task to delegate', 'type': 'str'}, 'context': {'description': 'The context for the task', 'type': 'str'}, 'coworker': {'description': 'The role/name of the coworker to delegate to', 'type': 'str'}}
Tool Description: Delegate a specific task to one of the following coworkers: Market Demand Analyst
The input to this tool should be the coworker, the task you want them to do, and ALL necessary context to execute 

ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyDH***************************0Ovk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


 Error during LLM call: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyDH***************************0Ovk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
 An unknown error occurred. Please check the details below.
 Error details: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyDH***************************0Ovk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
# Agent: Product Quality Inspector
## Task: 
    Inspect the product image at https://s.marketwatch.com/public/resources/images/MW-HT101_nerd_d_ZG_20191010165334.jpg.
    Identify any visible defects such as scratches, dents, misalignment

ERROR:root:LiteLLM call failed: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyDH***************************0Ovk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


 Error during LLM call: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyDH***************************0Ovk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
 An unknown error occurred. Please check the details below.
 Error details: litellm.AuthenticationError: AuthenticationError: OpenAIException - Error code: 401 - {'error': {'message': 'Incorrect API key provided: AIzaSyDH***************************0Ovk. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}


## Callbacks

In [39]:
from crewai import Agent

# AI Researcher Agent
research_agent = Agent(
    role="AI News Researcher",
    goal="Find and summarize the latest AI news from trusted sources",
    backstory="You are a dedicated AI journalist who follows the latest advancements in artificial intelligence.",
    verbose=False,
    llm=llm
)


In [40]:
def notify_team(output):

    print(f"""Task Completed!
              Task: {output.description}
              Output Summary: {output.summary}""")

    with open("latest_ai_news.txt", "w") as f:
        f.write(f"Task: {output.description}\n")
        f.write(f"Output Summary: {output.summary}\n")
        f.write(f"Full Output: {output.raw}\n")
    
    print("News summary saved to latest_ai_news.txt")

In [41]:
from crewai import Task

research_news_task = Task(
    description="Find and summarize the latest AI breakthroughs from the last week.",
    expected_output="A structured summary of AI news headlines.",
    agent=research_agent,
    callback=notify_team  # Calls the function after task completion
)


In [42]:
from crewai import Crew

ai_news_crew = Crew(
    agents=[research_agent],
    tasks=[research_news_task],
    verbose=False
)

# Execute the workflow
result = ai_news_crew.kickoff()


## Hierarchical Process

In [44]:
from crewai import Agent, LLM

llm = LLM(model="gemini/gemini-2.0-flash")

# research a new project idea, do the research on market demand, risk, and potential return on investment.

# Define the Manager AI
manager_agent = Agent(
    role="Project Research Manager",
    goal="Oversee the project research and ensure timely, high-quality responses.",
    backstory="""An experienced project manager responsible
                 for ensuring project research.""",
    allow_delegation=True,
    verbose=True,
    llm=llm
)

# Define the Technical Support AI
market_demand_agent = Agent(
    role="Market Demand Analyst",
    goal="Write market demand content.",
    backstory="""A skilled market demand analyst who
                 writes market demand content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
risk_analysis_agent = Agent(
    role="Risk Analysis Analyst",
    goal="Write risk analysis content.",
    backstory="""A risk analysis analyst who
                 writes fiction content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

# Define the Fiction Writer AI
return_on_investment_agent = Agent(
    role="Return on Investment Analyst",
    goal="Write return on investment content.",
    backstory="""A return on investment analyst who
                writes return on investment content.""",
    allow_delegation=False, 
    verbose=True,
    llm=llm
)


In [45]:
from crewai import Task

manager_task = Task(
    description="""Oversee the project research on {project_title} and ensure timely, high-quality responses.""",
    expected_output="A manager-approved response ready to be sent as an article on {project_title}.",
    agent=manager_agent, 
)

market_demand_task = Task(
    description="""Analyze the market demand for the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=market_demand_agent, 
)

risk_analysis_task = Task(
    description="""Analyze the risk of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=risk_analysis_agent, 
)

return_on_investment_task = Task(
    description="""Analyze the return on investment of the project title '{project_title}'""",
    expected_output="A categorized project title labeled as 'Technical' or 'Fiction'.",
    agent=return_on_investment_agent, 
)

final_report_task = Task(
    description="""Review the final responses from the 
                   market demand, risk analysis, and return on investment agents
                   and create a final report.""",
    expected_output="""A comprehensive report on the project title '{project_title}'
    containing the market demand, risk analysis, and return on investment.""",
    agent=manager_agent,
)

In [46]:
from crewai import Crew, Process

project_research_crew = Crew(
    agents=[market_demand_agent, risk_analysis_agent, return_on_investment_agent],
    
    tasks=[market_demand_task, risk_analysis_task, return_on_investment_task, final_report_task],
    
    manager_agent=manager_agent,
    
    process=Process.hierarchical,
    
    verbose=True,
)

In [47]:
inputs = {"project_title": "Multi-Agent System"}

result = project_research_crew.kickoff(inputs=inputs)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 2aca85cb-4633-4939-bf97-9d26fd189dab                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 2320116f-58f4-4e9b-b8ba-3742d03a87f5
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 2320116f-58f4-4e9b-b8ba-3742d03a87f5
       Status: Executing Task...
    └── 🤖 Agent: Project Research Manager
            Status: In Progress

🚀 Crew: crew
└── 📋 Task: 2320116f-58f4-4e9b-b8ba-3742d03a87f5
       Status: Executing Task...
    └── 🤖 Agent: Project Research Manager
            Status: In Progress
        └── 🧠 Thinking...

🚀 Crew: crew
└── 📋 Task: 2320116f-58f4-4e9b-b8ba-3742d03a87f5
       Status: Executing Task...
    └── 🤖 Agent: Project Research Manager
            Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 2320116f-58f4-4e9b-b8ba-3742d03a87f5
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Project Research Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
├── 📋 Task: 25df2e23-ac8a-4afb-87a2-f2ca421b4702
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
│       ├── 🔧 Failed Delegate work to coworker (3)
│       └── 🔧 Failed Ask question to coworker (1)
├── 📋 Task: d2fb5ce2-b826-42c5-b1f9-0ceed5a544cc
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
└── 📋 Task: 086ac1e4-26a3-4dac-9374-cddc314b7a10
       Status: Executing Task...
    ├── 🤖 Agent: Project Research Manager
    │       Status: In Progress
    │   ├── 🔧 Failed Delegate work to coworker (6)
    │   └── 🔧 Using Ask question to coworker (2)
    └── 🤖 Agent: Project Research Manager
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 2320116f-58f4-4e9b-b8ba-3742d03a87f5
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Project Research Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
├── 📋 Task: 25df2e23-ac8a-4afb-87a2-f2ca421b4702
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
│       ├── 🔧 Failed Delegate work to coworker (3)
│       └── 🔧 Failed Ask question to coworker (1)
├── 📋 Task: d2fb5ce2-b826-42c5-b1f9-0ceed5a544cc
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
└── 📋 Task: 086ac1e4-26a3-4dac-9374-cddc314b7a10
       Status: Executing Task...
    ├── 🤖 Agent: Project Research Manager
    │       Status: In Progress
    │   ├── 🔧 Failed Delegate work to coworker (6)
    │   └── 🔧 Using Ask question to coworker (2)
    └── 🤖 Agent: Project Research Manager
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 2320116f-58f4-4e9b-b8ba-3742d03a87f5
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   ├── 🤖 Agent: Project Research Manager
│   │       Status: In Progress
│   │   └── 🔧 Using Delegate work to coworker (2)
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
├── 📋 Task: 25df2e23-ac8a-4afb-87a2-f2ca421b4702
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
│       ├── 🔧 Failed Delegate work to coworker (3)
│       └── 🔧 Failed Ask question to coworker (1)
├── 📋 Task: d2fb5ce2-b826-42c5-b1f9-0ceed5a544cc
│      Assigned to: Project Research Manager
│      Status: ✅ Completed
│   └── 🤖 Agent: Project Research Manager
│           Status: ✅ Completed
└── 📋 Task: 086ac1e4-26a3-4dac-9374-cddc314b7a10
       Assigned to: Project Research Manager
       Status: ✅ Completed
    ├── 🤖 Agent: Project Research Manager
    │       Status: In Progress
    │   ├── 🔧 Failed Delegate work to coworker (6)
    │   └── 🔧 Using Ask question to coworker (2)
    └── 🤖 Agent: Project Research Manager
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 086ac1e4-26a3-4dac-9374-cddc314b7a10                                                                     │
│  Agent: Project Research Manager                                                                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 2aca85cb-4633-4939-bf97-9d26fd189dab                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [48]:
from IPython.display import Markdown
Markdown(result.raw)

```
**Multi-Agent System Project Reports**

**I. Market Demand Report**

**1. Executive Summary:**

This report assesses the market demand for Multi-Agent Systems (MAS) across various sectors. The analysis indicates a growing demand driven by increasing complexity in business operations, the need for automation, and advancements in artificial intelligence. Key industries showing significant interest include manufacturing, logistics, healthcare, finance, and cybersecurity.

**2. Market Overview:**

*   **Definition:** Multi-Agent Systems are decentralized systems composed of multiple intelligent agents that interact to solve problems that are beyond the capabilities of individual agents or centralized systems.
*   **Market Size and Growth:** The global MAS market is currently valued at \$X billion (replace with a realistic estimate, e.g., \$5 billion) and is projected to reach \$Y billion (e.g., \$15 billion) by 2030, exhibiting a CAGR of Z% (e.g., 15%) during the forecast period.
*   **Key Drivers:**
    *   Increasing adoption of AI and machine learning.
    *   Rising need for automation and optimization in various industries.
    *   Growing complexity of business processes.
    *   Advancements in computing power and cloud technologies.
*   **Restraints:**
    *   High initial investment costs.
    *   Complexity in designing and implementing MAS.
    *   Lack of skilled professionals.
    *   Concerns regarding security and data privacy.

**3. Market Segmentation:**

*   **By Application:**
    *   **Manufacturing:** Supply chain management, process optimization, predictive maintenance.
    *   **Logistics:** Route optimization, warehouse management, autonomous delivery systems.
    *   **Healthcare:** Patient monitoring, disease diagnosis, drug discovery.
    *   **Finance:** Fraud detection, algorithmic trading, risk management.
    *   **Cybersecurity:** Threat detection, intrusion prevention, vulnerability assessment.
    *   **Robotics:** Multi-robot coordination, collaborative tasks.
    *   **Energy:** Smart grids, energy management, resource optimization.
*   **By Deployment Model:**
    *   On-premise
    *   Cloud-based
*   **By Region:**
    *   North America
    *   Europe
    *   Asia Pacific
    *   Latin America
    *   Middle East & Africa

**4. Competitive Landscape:**

*   **Key Players:** (List major companies and research institutions involved in MAS development and deployment. Example:)
    *   IBM
    *   Microsoft
    *   Google
    *   Amazon
    *   [Specific MAS Solution Providers]
*   **Competitive Advantages:** (Examples)
    *   Proprietary algorithms and AI models.
    *   Strong domain expertise.
    *   Established partnerships with industry leaders.
    *   Scalable and customizable solutions.

**5. Market Trends:**

*   **Integration with IoT:** Combining MAS with Internet of Things (IoT) devices to enable real-time decision-making and automation.
*   **Edge Computing:** Deploying MAS at the edge of the network to reduce latency and improve performance.
*   **Explainable AI (XAI):** Focus on developing MAS with transparent and explainable decision-making processes.
*   **Autonomous Agents:** Increasing use of autonomous agents capable of learning and adapting to changing environments.

**6. Market Demand Forecast:**

| Application     | 2024 (USD Million) | 2030 (USD Million) | CAGR (%) |
| :---------------- | :----------------- | :----------------- | :------- |
| Manufacturing     | X1                 | Y1                 | Z1       |
| Logistics         | X2                 | Y2                 | Z2       |
| Healthcare        | X3                 | Y3                 | Z3       |
| Finance           | X4                 | Y4                 | Z4       |
| Cybersecurity     | X5                 | Y5                 | Z5       |
| **Total**         | **XT**             | **YT**             | **ZT**    |

*(Replace X, Y, Z values with reasonable estimations.)*

**7. Conclusion:**

The market for Multi-Agent Systems is poised for significant growth in the coming years. Organizations that adopt MAS solutions can gain a competitive advantage by improving efficiency, reducing costs, and enabling innovation. Focus on addressing the challenges related to complexity, security, and skills gap will be crucial for realizing the full potential of MAS.

**II. Risk Analysis Report**

**1. Executive Summary:**

This report identifies and assesses the potential risks associated with the Multi-Agent System project. Risks are categorized into technical, operational, financial, and market-related areas. Mitigation strategies are proposed to minimize the impact of these risks.

**2. Risk Identification:**

| Risk Category | Risk Description                                                              | Likelihood | Impact | Severity (Likelihood x Impact) |
| :------------ | :---------------------------------------------------------------------------- | :--------- | :----- | :----------------------------- |
| Technical     | Complexity of system design and integration.                                  | High       | High   | High                           |
| Technical     | Difficulty in ensuring agent coordination and communication.                    | Medium     | High   | Medium                         |
| Technical     | Scalability issues as the number of agents increases.                           | Medium     | Medium | Medium                         |
| Technical     | Integration with legacy systems poses integration and compatibility challenges. | Medium     | High   | Medium                         |
| Operational   | Lack of skilled personnel to develop and maintain the MAS.                   | Medium     | High   | Medium                         |
| Operational   | Data privacy and security breaches.                                             | Medium     | High   | Medium                         |
| Financial     | Cost overruns due to unforeseen complexities.                                | Low        | High   | Medium                         |
| Financial     | Budget cuts or funding delays.                                                | Low        | Medium | Low                            |
| Market        | Changes in market demand or competitive landscape.                             | Low        | Medium | Low                            |
| Market        | Resistance to adoption from end-users.                                        | Low        | Medium | Low                            |

*Likelihood: Low, Medium, High*

*Impact: Low, Medium, High*

*Severity: Risk Score based on Likelihood x Impact.*

**3. Risk Assessment:**

*   **Technical Risks:** The high complexity of MAS design and integration poses a significant risk. Ensuring seamless communication and coordination between agents requires careful planning and robust testing.
*   **Operational Risks:** A shortage of skilled professionals with expertise in MAS development and maintenance could delay the project and increase costs. Data privacy and security breaches are also a major concern, requiring robust security measures.
*   **Financial Risks:** Cost overruns and budget cuts could impact the project's scope and timeline.
*   **Market Risks:** Changes in market demand and resistance from end-users could affect the project's success.

**4. Mitigation Strategies:**

| Risk Description                                               | Mitigation Strategy                                                                                                                      |
| :------------------------------------------------------------- | :--------------------------------------------------------------------------------------------------------------------------------------- |
| Complexity of system design and integration.                   | Adopt modular design principles, use established MAS frameworks, conduct thorough testing and simulation.                                 |
| Difficulty in ensuring agent coordination and communication.    | Implement robust communication protocols, use agent-oriented programming languages, conduct extensive simulations to validate coordination. |
| Scalability issues.                                            | Design the system with scalability in mind, use cloud-based infrastructure, implement load balancing techniques.                          |
| Lack of skilled personnel.                                     | Invest in training and development programs, hire experienced MAS developers, partner with research institutions.                           |
| Data privacy and security breaches.                            | Implement robust security measures, encrypt sensitive data, comply with relevant data privacy regulations.                                 |
| Cost overruns.                                                 | Develop a detailed budget, implement cost control measures, monitor project spending closely.                                             |
| Changes in market demand.                                      | Conduct market research, monitor industry trends, adapt the project to meet changing market needs.                                       |
| Integration with legacy systems poses compatibility challenges. | Use API, design adapters, conduct extensive testing.                                                                                    |

**5. Risk Monitoring and Control:**

*   Establish a risk management plan.
*   Regularly monitor and review risks.
*   Update the risk register as needed.
*   Implement corrective actions to mitigate risks.

**6. Conclusion:**

Effective risk management is essential for the success of the Multi-Agent System project. By identifying, assessing, and mitigating potential risks, the project team can minimize disruptions and ensure that the project achieves its objectives.

**III. Return on Investment (ROI) Report**

**1. Executive Summary:**

This report assesses the potential return on investment (ROI) for the Multi-Agent System project. The analysis considers the project's costs, benefits, and payback period. The results indicate a positive ROI, making the project financially viable.

**2. Project Costs:**

| Cost Category          | Amount (USD) |
| :--------------------- | :----------- |
| Development Costs      | X1           |
| Infrastructure Costs   | X2           |
| Personnel Costs        | X3           |
| Training Costs         | X4           |
| Maintenance Costs      | X5           |
| **Total Costs**        | **XT**       |

*(Replace X values with reasonable estimations.)*

**3. Project Benefits:**

| Benefit Category       | Description                                                                   | Year 1 (USD) | Year 2 (USD) | Year 3 (USD) | Year 4 (USD) | Year 5 (USD) |
| :--------------------- | :---------------------------------------------------------------------------- | :----------- | :----------- | :----------- | :----------- | :----------- |
| Increased Efficiency   | Automation of tasks, improved decision-making.                              | B1_Y1        | B1_Y2        | B1_Y3        | B1_Y4        | B1_Y5        |
| Reduced Costs          | Lower labor costs, reduced waste, optimized resource utilization.             | B2_Y1        | B2_Y2        | B2_Y3        | B2_Y4        | B2_Y5        |
| Improved Productivity  | Faster processing times, increased throughput.                               | B3_Y1        | B3_Y2        | B3_Y3        | B3_Y4        | B3_Y5        |
| Enhanced Decision Making | Better insights, data-driven decisions.                                      | B4_Y1        | B4_Y2        | B4_Y3        | B4_Y4        | B4_Y5        |
| New Revenue Streams   | New products or services enabled by the MAS.                                | B5_Y1        | B5_Y2        | B5_Y3        | B5_Y4        | B5_Y5        |
| **Total Benefits**     |                                                                               | **BT_Y1**    | **BT_Y2**    | **BT_Y3**    | **BT_Y4**    | **BT_Y5**    |

*(Replace B and Y values with reasonable estimations.)*

**4. ROI Calculation:**

*   **Total Investment:** XT (from Project Costs)
*   **Total Benefits (5 years):** Sum of BT_Y1 to BT_Y5
*   **Net Benefit:** Total Benefits - Total Investment
*   **ROI:** (Net Benefit / Total Investment) * 100

    *Example:*

    *   Total Investment = \$1,000,000
    *   Total Benefits = \$3,000,000
    *   Net Benefit = \$2,000,000
    *   ROI = (2,000,000 / 1,000,000) * 100 = 200%

*   **Payback Period:** The time it takes for the project to recoup its initial investment. Calculated by dividing the initial investment by the annual net benefit.

**5. Sensitivity Analysis:**

*   Assess the impact of changes in key assumptions (e.g., discount rate, benefit estimates) on the ROI.
*   Identify the factors that have the greatest impact on the project's financial viability.

**6. Conclusion:**

The Multi-Agent System project is expected to generate a positive ROI and a reasonable payback period. The project's financial viability is dependent on realizing the projected benefits and managing costs effectively. Continuous monitoring and evaluation of the project's financial performance are essential for ensuring its long-term success.

**Disclaimer:** These reports are based on simulated data and general industry knowledge. Actual results may vary. A detailed feasibility study and financial analysis should be conducted to validate these findings.
```

## Human input

In [49]:
from crewai import Agent, LLM

llm = LLM(model="gemini/gemini-2.0-flash")

# AI Researcher Agent
researcher_agent = Agent(
    role="Senior AI Researcher",
    goal="Discover and summarize the latest trends in AI and technology.",
    backstory="An expert in AI research who tracks emerging trends and their real-world applications.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# AI Content Strategist Agent
content_strategist_agent = Agent(
    role="Tech Content Strategist",
    goal="Transform AI research insights into compelling blog content.",
    backstory="An experienced tech writer who makes AI advancements accessible to a broad audience.",
    verbose=True,
    allow_delegation=True,
    llm=llm
)

In [50]:
from crewai import Task

# Step 1: AI Research with Human Oversight
ai_research_task = Task(
    description=(
        "Conduct a deep analysis of AI trends in 2025. Identify key innovations, breakthroughs, and market shifts. "
        "Before finalizing, ask a human reviewer for feedback to refine the report."
    ),
    expected_output="A structured research summary covering AI advancements in 2025.",
    agent=researcher_agent,
    human_input=True  # Requires human validation before finalizing the answer
)

# Step 2: AI-Generated Blog Post with Human Review
blog_post_task = Task(
    description=(
        "Using insights from the AI Researcher, create an engaging blog post summarizing key AI advancements. "
        "Ensure the post is informative and accessible. Before finalizing, ask a human reviewer for approval."
    ),
    expected_output="A well-structured blog post about AI trends in 2025.",
    agent=content_strategist_agent,
    human_input=True  # Requires human approval before publishing
)

In [51]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[researcher_agent, content_strategist_agent],  
    tasks=[ai_research_task, blog_post_task],  
    verbose=True,  
)


In [52]:
# Execute the workflow
result = ai_research_crew.kickoff()

# Display the final validated research output
print("\n=== Final AI Research Report ===")
print(result.raw)


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: aff73d94-0b97-4ac8-8567-3ec8ec95256c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 49a93cd9-02ae-4f18-ab61-6b4de90f869b
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 49a93cd9-02ae-4f18-ab61-6b4de90f869b
       Status: Executing Task...
    └── 🤖 Agent: Senior AI Researcher
            Status: In Progress

🚀 Crew: crew
└── 📋 Task: 49a93cd9-02ae-4f18-ab61-6b4de90f869b
       Status: Executing Task...
    └── 🤖 Agent: Senior AI Researcher
            Status: In Progress
        └── 🧠 Thinking...

🚀 Crew: crew
└── 📋 Task: 49a93cd9-02ae-4f18-ab61-6b4de90f869b
       Status: Executing Task...
    └── 🤖 Agent: Senior AI Researcher
            Status: In Progress

...

🚀 Crew: crew
├── 📋 Task: 49a93cd9-02ae-4f18-ab61-6b4de90f869b
│      Assigned to: Senior AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior AI Researcher
│           Status: ✅ Completed
└── 📋 Task: 6a525747-0509-4194-8f7e-a0bbd32f34ff
       Status: Executing Task...
    ├── 🤖 Agent: Tech Content Strategist
    │       Status: In Progress
    │   └── 🔧 Using Ask question to coworker (3)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (7)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (8)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Ask question to coworker (4)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (9)
    └── 🤖 Agent: Senior AI Researcher
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 49a93cd9-02ae-4f18-ab61-6b4de90f869b
│      Assigned to: Senior AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior AI Researcher
│           Status: ✅ Completed
└── 📋 Task: 6a525747-0509-4194-8f7e-a0bbd32f34ff
       Status: Executing Task...
    ├── 🤖 Agent: Tech Content Strategist
    │       Status: In Progress
    │   └── 🔧 Using Ask question to coworker (3)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (7)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (8)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Ask question to coworker (4)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (9)
    └── 🤖 Agent: Tech Content Strategist
            Status: ✅ Completed

🚀 Crew: crew
├── 📋 Task: 49a93cd9-02ae-4f18-ab61-6b4de90f869b
│      Assigned to: Senior AI Researcher
│      Status: ✅ Completed
│   └── 🤖 Agent: Senior AI Researcher
│           Status: ✅ Completed
└── 📋 Task: 6a525747-0509-4194-8f7e-a0bbd32f34ff
       Assigned to: Tech Content Strategist
       Status: ✅ Completed
    ├── 🤖 Agent: Tech Content Strategist
    │       Status: In Progress
    │   └── 🔧 Using Ask question to coworker (3)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (7)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (8)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Ask question to coworker (4)
    ├── 🤖 Agent: Senior AI Researcher
    │       Status: ✅ Completed
    │   └── 🔧 Using Delegate work to coworker (9)
    └── 🤖 Agent: Tech Content Strategist
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6a525747-0509-4194-8f7e-a0bbd32f34ff                                                                     │
│  Agent: Tech Content Strategist                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: aff73d94-0b97-4ac8-8567-3ec8ec95256c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
from IPython.display import Markdown
Markdown(result.raw)